In [ ]:
# ==============================================================================
# [실습 2] Agent Framework 비교: smolagents vs LangChain
# ==============================================================================
#
# 목표: "미니 주식 애널리스트" 에이전트를 3가지 방식으로 구현해보고 장단점을 비교합니다.
#
# [미션]
# "테슬라(TSLA)의 현재 주가와 최신 뉴스를 검색한 뒤,
#  이 정보를 바탕으로 간단한 투자 의견(매수/매도/보류) 리포트를 작성하라."
#
# [비교 포인트]
# 1. smolagents (CodeAgent): LLM이 Python 코드를 직접 작성하여 실행 (강력함)
# 2. LangGraph (Graph): 노드와 엣지로 흐름을 제어 (정교함)
# 3. smolagents (ToolCallingAgent): JSON 도구 호출 방식 (안전함 & 간결함) <- 추천!
#
# [준비물]
# - OpenAI API Key
# ==============================================================================


In [ ]:
# 1. 라이브러리 설치 (최초 1회 실행)
# !pip install smolagents langchain langchain-openai langgraph yfinance duckduckgo-search openai

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     ------------- -------------------------- 1.0/3.0 MB 2.1 MB/s eta 0:00:01
     ------------- -------------------------- 1.0/3.0 MB 2.1 MB/s eta 0:00:01
     ------------- -------------------------- 1.0/3.0 MB 2.1 MB/s eta 0:00:01
     -------------------- ------------------- 1.6/3.0 MB 1.3 MB/s eta 0:00:02
     --------------------------- ------------ 2.1/3.0 MB 1.5 MB/s eta 0:00:01
     ------------------------------- -------- 2.4/3.0 MB 1.5 MB/s eta 0

  You can safely remove it manually.


In [ ]:
import os
import yfinance as yf
from duckduckgo_search import DDGS

# API 키 설정
# os.environ["OPENAI_API_KEY"] = "sk-..." 

if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️ 경고: OPENAI_API_KEY가 설정되지 않았습니다.")

# ==============================================================================
# [Common] 공통 도구(Tool) 로직 정의
# ==============================================================================
# 모든 프레임워크에서 공통으로 사용할 핵심 기능 함수입니다.

def get_stock_price_logic(ticker: str) -> str:
    """주식 티커(예: TSLA, AAPL)를 받아 현재 주가를 반환합니다."""
    try:
        ticker = ticker.upper()
        stock = yf.Ticker(ticker)
        # 최근 1일치 데이터 조회
        history = stock.history(period="1d")
        if history.empty:
            return f"{ticker}에 대한 데이터를 찾을 수 없습니다."
        price = history['Close'].iloc[-1]
        return f"{ticker}의 현재 주가는 ${price:.2f} 입니다."
    except Exception as e:
        return f"주가 조회 실패: {str(e)}"

def web_search_logic(query: str) -> str:
    """웹 검색을 수행하여 최신 뉴스 요약을 반환합니다."""
    try:
        results = DDGS().text(query, max_results=3)
        if not results:
            return "검색 결과가 없습니다."
        summary = "\n".join([f"- {r['title']}: {r['body']}" for r in results])
        return summary
    except Exception as e:
        return f"검색 실패: {str(e)}"

print("✅ 공통 도구 로직 준비 완료")


# ==============================================================================
# [Case 1] smolagents (CodeAgent) - "강력한 코드 실행"
# ==============================================================================
# 특징: 에이전트가 "생각" 후 "Python 코드"를 짜서 실행합니다.
# 장점: 데이터 가공, 계산, 반복문 등을 자유롭게 사용 가능.
# 단점: 임의의 코드 실행으로 보안 리스크 존재 (Sandbox 필수).
# ==============================================================================

from smolagents import CodeAgent, Tool, LiteLLMModel

# 도구 래핑 (smolagents 방식)
class StockPriceTool(Tool):
    name = "get_stock_price"
    description = "주식 티커를 입력받아 현재 주가를 반환합니다."
    inputs = {"ticker": {"type": "string", "description": "주식 티커 (예: TSLA)"}}
    output_type = "string"
    def forward(self, ticker: str) -> str:
        return get_stock_price_logic(ticker)

class WebSearchTool(Tool):
    name = "web_search"
    description = "검색어를 입력받아 웹 검색 결과를 반환합니다."
    inputs = {"query": {"type": "string", "description": "검색어"}}
    output_type = "string"
    def forward(self, query: str) -> str:
        return web_search_logic(query)

# 모델 설정 (OpenAI gpt-4.1 사용)
model = LiteLLMModel(model_id="gpt-4.1", temperature=0.0)

# CodeAgent 생성
agent_code = CodeAgent(
    tools=[StockPriceTool(), WebSearchTool()],
    model=model,
    add_base_tools=True # print 등 기본 파이썬 도구 허용
)

print("\n🤖 [Case 1] smolagents (CodeAgent) 실행...")
task_query = "테슬라(TSLA) 주가와 뉴스를 보고 한글로 투자 의견을 요약해줘."

# 실행
result_code = agent_code.run(task_query)
print("\n📝 [CodeAgent 결과]:", result_code)



✅ 공통 도구 로직 준비 완료

🤖 [Case 1] smolagents (CodeAgent) 실행...


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 테슬라(TSLA) 주가와 뉴스를 보고 한글로 투자 의견을 요약해줘.                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1 ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  price = get_stock_price("TSLA")                                                                                  
  print(f"TSLA 현재 주가: {price}")                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
TSLA 현재 주가: TSLA의 현재 주가는 $454.53 입니다.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  opinion = (                                                                                                      
      "테슬라(TSLA)의 현재 주가는 $454.53입니다. 최근 미국 정부의 로봇산업 및 AI 개발 가속화 정책에 힘입어 주가가  
  상승하고 있습니다. "                                                                                             
      "또한 자율주행 기술 도입, 신차 출시, 다양한 가격 정책 등 긍정적인 이슈가 이어지고 있습니다. "                
      "다만, 국내 판매 차량의 자율주행 기능 제한 등 일부 리스크도 존재합니다. "                                    
      "전반적으로 성장 동력이 강하나, 단기 급등에 따른 변동성에 유의하며 중장기적 관점에서 분할 매수 전략이        
  유효해 보입니다."                                                                                                
  )                                                                                                                
  final_answer(opinion)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 테슬라(TSLA)의 현재 주가는 $454.53입니다. 최근 미국 정부의 로봇산업 및 AI 개발 가속화 정책에 
힘입어 주가가 상승하고 있습니다. 또한 자율주행 기술 도입, 신차 출시, 다양한 가격 정책 등 긍정적인 이슈가 이어지고 
있습니다. 다만, 국내 판매 차량의 자율주행 기능 제한 등 일부 리스크도 존재합니다. 전반적으로 성장 동력이 강하나, 
단기 급등에 따른 변동성에 유의하며 중장기적 관점에서 분할 매수 전략이 유효해 보입니다.

[Step 3: Duration 9.33 seconds| Input tokens: 7,773 | Output tokens: 391]


📝 [CodeAgent 결과]: 테슬라(TSLA)의 현재 주가는 $454.53입니다. 최근 미국 정부의 로봇산업 및 AI 개발 가속화 정책에 힘입어 주가가 상승하고 있습니다. 또한 자율주행 기술 도입, 신차 출시, 다양한 가격 정책 등 긍정적인 이슈가 이어지고 있습니다. 다만, 국내 판매 차량의 자율주행 기능 제한 등 일부 리스크도 존재합니다. 전반적으로 성장 동력이 강하나, 단기 급등에 따른 변동성에 유의하며 중장기적 관점에서 분할 매수 전략이 유효해 보입니다.


In [3]:

# ==============================================================================
# [Case 2] LangGraph - "정교한 그래프 제어"
# ==============================================================================
# 특징: 상태(State)와 노드(Node)로 흐름을 정의.
# 장점: 엔터프라이즈급 안정성, 실행 흐름의 완벽한 통제.
# 단점: 설정해야 할 코드(Boilerplate)가 많고 복잡함.
# ==============================================================================

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

print("\n" + "-"*50)
print("🤖 [Case 2] LangGraph 실행...")

# 도구 래핑 (LangChain @tool 데코레이터)
@tool
def get_stock_price(ticker: str) -> str:
    """주식 티커(예: TSLA)를 받아 현재 주가를 조회합니다."""
    return get_stock_price_logic(ticker)

@tool
def web_search(query: str) -> str:
    """웹 검색을 수행하여 정보를 찾습니다."""
    return web_search_logic(query)

tools = [get_stock_price, web_search]

# 모델 및 그래프 생성
llm = ChatOpenAI(model="gpt-4.1", temperature=0.0)
agent_graph = create_react_agent(llm, tools)

# 실행
print("🔄 그래프 실행 중...")
response = agent_graph.invoke({"messages": [HumanMessage(content=task_query)]})
print("\n📝 [LangGraph 결과]:", response["messages"][-1].content)




--------------------------------------------------
🤖 [Case 2] LangGraph 실행...


C:\Users\user\AppData\Local\Temp\ipykernel_7420\878787352.py:32: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_graph = create_react_agent(llm, tools)


🔄 그래프 실행 중...

📝 [LangGraph 결과]: 테슬라(TSLA) 주가와 최근 뉴스를 바탕으로 투자 의견을 요약해드리겠습니다.

1. 주가 현황
- 테슬라(TSLA)의 현재 주가는 $454.53입니다.

2. 최근 뉴스 요약
- 테슬라는 2025년형 모델별 트림, 옵션, 가격 정보를 정리해 공개했습니다. 모델 Y, 3, S, X의 국내 판매 가격과 옵션 구성이 한눈에 보기 쉽게 정리되었습니다.
- 2025년형 테슬라 전기차는 가격 대비 성능, 기술력, 브랜드 신뢰도 면에서 여전히 시장에서 경쟁력이 높다는 평가를 받고 있습니다.
- 모델 3와 Y는 실용성과 가격 경쟁력이, S와 X는 고급스러움과 퍼포먼스가 강점으로 꼽힙니다.

3. 투자 의견 (한글 요약)
- 테슬라는 전기차 시장에서 여전히 선두를 유지하고 있으며, 다양한 모델과 가격 정책으로 소비자층을 넓히고 있습니다.
- 기술력과 브랜드 신뢰도가 높아 중장기적으로 긍정적인 전망이 우세합니다.
- 다만, 전기차 시장의 경쟁 심화와 글로벌 경기 변동성, 정책 변화 등은 주가 변동 요인으로 작용할 수 있으니 투자 시 유의가 필요합니다.

종합적으로, 테슬라는 성장성과 시장 지배력을 바탕으로 중장기 투자에 긍정적인 종목으로 평가할 수 있습니다. 단, 단기 변동성에 주의하며 분산 투자와 리스크 관리가 필요합니다.


In [4]:

# ==============================================================================
# [Case 3] smolagents (ToolCallingAgent) - "안전하고 간결함" (추천!)
# ==============================================================================
# 특징: CodeAgent와 사용법은 비슷하지만, 내부적으로는 JSON 도구 호출(Tool Calling)을 사용.
# 장점:
#   1. 보안성: 임의의 파이썬 코드를 실행하지 않음 (LangGraph와 동일하게 안전).
#   2. 간결성: LangGraph보다 코드가 훨씬 짧고 직관적임 (smolagents의 장점).
# ==============================================================================

from smolagents import ToolCallingAgent

print("\n" + "-"*50)
print("🤖 [Case 3] smolagents (ToolCallingAgent) 실행...")
print("   👉 포인트: 안전한 JSON 호출 방식을 쓰면서도 코드는 매우 짧습니다!")

# ToolCallingAgent 생성 (도구는 위에서 만든 것 재사용)
agent_toolcall = ToolCallingAgent(
    tools=[StockPriceTool(), WebSearchTool()],
    model=model
)

# 실행
result_toolcall = agent_toolcall.run(task_query)
print("\n📝 [ToolCallingAgent 결과]:", result_toolcall)




--------------------------------------------------
🤖 [Case 3] smolagents (ToolCallingAgent) 실행...
   👉 포인트: 안전한 JSON 호출 방식을 쓰면서도 코드는 매우 짧습니다!


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 테슬라(TSLA) 주가와 뉴스를 보고 한글로 투자 의견을 요약해줘.                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1 ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_stock_price' with arguments: {'ticker': 'TSLA'}                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': '테슬라 TSLA 최신 뉴스'}                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '현재 테슬라(TSLA)의 주가는 $454.53입니다. 최신 뉴스에  │
│ 따르면 2025년형 테슬라 전기차는 가격 대비 성능, 기술력, 브랜드 신뢰도 면에서 여전히 최상위권에 속하며, 모델 3와 │
│ Y는 실용성과 가격경쟁력이, S와 X는 고급스러움과 퍼포먼스가 강점으로 평가받고 있습니다.\n\n투자 의견             │
│ 요약:\n테슬라는 전기차 시장에서의 기술력과 브랜드 신뢰도를 바탕으로 여전히 경쟁력이 높으며, 다양한 모델         │
│ 라인업과 가격 경쟁력도 유지하고 있습니다. 다만, 주가가 이미 상당히 높은 수준이므로 추가 상승 여력과 시장        │
│ 변동성, 경쟁사 동향 등을 함께 고려해 신중한 접근이 필요합니다. 장기적으로는 긍정적이지만, 단기 변동성에         │
│ 유의하는 것이 좋겠습니다.'}                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 현재 테슬라(TSLA)의 주가는 $454.53입니다. 최신 뉴스에 따르면 2025년형 테슬라 전기차는 가격 대비 성능,
기술력, 브랜드 신뢰도 면에서 여전히 최상위권에 속하며, 모델 3와 Y는 실용성과 가격경쟁력이, S와 X는 고급스러움과 
퍼포먼스가 강점으로 평가받고 있습니다.

투자 의견 요약:
테슬라는 전기차 시장에서의 기술력과 브랜드 신뢰도를 바탕으로 여전히 경쟁력이 높으며, 다양한 모델 라인업과 가격 
경쟁력도 유지하고 있습니다. 다만, 주가가 이미 상당히 높은 수준이므로 추가 상승 여력과 시장 변동성, 경쟁사 동향 등을
함께 고려해 신중한 접근이 필요합니다. 장기적으로는 긍정적이지만, 단기 변동성에 유의하는 것이 좋겠습니다.

[Step 2: Duration 5.73 seconds| Input tokens: 2,543 | Output tokens: 270]


📝 [ToolCallingAgent 결과]: 현재 테슬라(TSLA)의 주가는 $454.53입니다. 최신 뉴스에 따르면 2025년형 테슬라 전기차는 가격 대비 성능, 기술력, 브랜드 신뢰도 면에서 여전히 최상위권에 속하며, 모델 3와 Y는 실용성과 가격경쟁력이, S와 X는 고급스러움과 퍼포먼스가 강점으로 평가받고 있습니다.

투자 의견 요약:
테슬라는 전기차 시장에서의 기술력과 브랜드 신뢰도를 바탕으로 여전히 경쟁력이 높으며, 다양한 모델 라인업과 가격 경쟁력도 유지하고 있습니다. 다만, 주가가 이미 상당히 높은 수준이므로 추가 상승 여력과 시장 변동성, 경쟁사 동향 등을 함께 고려해 신중한 접근이 필요합니다. 장기적으로는 긍정적이지만, 단기 변동성에 유의하는 것이 좋겠습니다.
